# Prédictions des années de plantations des arbres de Grenoble

Bienvenue dans le notebook prédiction des années de plantations des arbres de Grenoble. Les données utilisées sont des informations recueillies sur les arbres plantés à Grenoble de ... 

## Objectifs :
- Analyser les données des arbres de Grenoble
- Construire plusieurs modèles de machine learning pour prédire les années de plantation de ces  arbres de Grenoble.
- Faire une évaluation robuste grâce à la cross validation.
- Améliorer leurs performances grâce à la Grid search
- Comparer ces modèles entre elles 

### Étape 1 : Importation des modules requise pour l'étude

In [2]:
import pandas as pd 

### Étape 2 : Chargement des données 

In [11]:
df_arbres = pd.read_csv("data/arbres_grenoble_epsg4326.csv")
df_arbres.head()

,elem_point_id,code,nom,genre,genre_desc,categorie,categorie_desc,sous_categorie,sous_categorie_desc,code_parent,...,diametrearbre,causeabattage,collectivite,stationmetro,hauteurarbre,portarbre,structure,forme,typenature,geo_point_2d
0,25301,ESP30399,AR30399,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,...,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.164519466065315,5.7364309029098335"
1,17909,ESP5185,AR5185,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,...,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.16450876797978,5.736451624755175"
2,28960,ESP16027,AR16027,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,...,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.164556703715526,5.736307688353201"
3,14246,ESP17702,AR17702,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP761,...,NaN,NaN,Ville de Grenoble,NaN,NaN,NaN,Ville de Grenoble,NaN,NaN,"45.164533444541846,5.736391508663688"
4,45677,ESP41299,AR41299,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45.18330787379752,5.709045164969469"


### Étape 3 : Prétraitement des données

In [20]:
    df_arbres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31895 entries, 0 to 31894
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   elem_point_id          31895 non-null  int64  
 1   code                   31895 non-null  object 
 2   nom                    31895 non-null  object 
 3   genre                  31895 non-null  object 
 4   genre_desc             31895 non-null  object 
 5   categorie              31895 non-null  object 
 6   categorie_desc         31895 non-null  object 
 7   sous_categorie         31895 non-null  object 
 8   sous_categorie_desc    31895 non-null  object 
 9   code_parent            31895 non-null  object 
 10  code_parent_desc       31895 non-null  object 
 11  adr_secteur            31895 non-null  int64  
 12  bien_reference         31895 non-null  object 
 13  genre_bota             31079 non-null  object 
 14  espece                 29415 non-null  object 
 15  va

On supprime toutes les colonnes ne possédant aucune informations ou possédant très peu d'informations

In [30]:
def high_nan_Columns_list(df, seuil):
    
    """Retourne une liste contenant des une proportion de valeur manquante superieur a un seuil."""
    
    return [elm for elm in df.columns if (df[elm].isna().sum()/df.shape[0])*100 >= seuil] 


list90nan = high_nan_Columns_list(df_arbres, 90)

# suppresion des  colonnes de plus de 90% de valeurs manquantes

df_arbres = df_arbres.drop(list90nan, axis = 1)

Maintenant Examinons le nombre valeurs uniques par colonnes

In [31]:
df_arbres.nunique()

elem_point_id           31895
code                    31895
nom                     31895
genre                       1
genre_desc                  1
categorie                   1
categorie_desc              1
sous_categorie              4
sous_categorie_desc         4
code_parent              1506
code_parent_desc         1506
adr_secteur                11
bien_reference          31895
genre_bota                120
espece                    292
variete                   222
stadededeveloppement        3
remarques                3020
anneedeplantation          68
raisondeplantation          3
typeimplantationplu         4
collectivite                2
hauteurarbre                3
portarbre                   7
structure                   4
forme                       3
typenature                  7
geo_point_2d            31895
dtype: int64